# Fake News Classifier Using LSTM

In [2]:
import pandas as pd

In [5]:
df = pd.read_csv('C:/Users/User/Desktop/19Nov/21/NLP/train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


We are using 'title' as independent variable and predictiing 'label' which shows it is fake news or not.

In [7]:
#Drop na values
df = df.dropna()

In [9]:
df.head(10)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0
10,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that...",0
11,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...,0


In [10]:
df.reset_index(inplace=True)

In [11]:
df.head(10)

,index,id,title,author,text,label
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
6,7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
7,9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0
8,10,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that...",0
9,11,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...,0


In [12]:
# Get independent and dependent variable

X = df.drop('label', axis=1)
y = df['label']

In [13]:
print(X.shape)
print(y.shape)

(18285, 5)
(18285,)


In [14]:
msg = X.copy()

# 1. Data Preprocessing 

In [16]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [17]:
#whole data title is cleaned - lower, remove punc, split, stemmed.
ps = PorterStemmer()
corpus = []

for i in range(len(msg)):
    review = re.sub('[^a-zA-Z]',' ',msg['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [18]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

# 2. One hot representation

In [19]:
from tensorflow.keras.preprocessing.text import one_hot

In [20]:
vocab_size=5000

In [21]:
onehot_repr = [one_hot(words, vocab_size) for words in corpus]
onehot_repr

[[3707, 3742, 1442, 4891, 1280, 1794, 3427, 341, 667, 1629],
 [4345, 3955, 4821, 1375, 2410, 2109, 1703],
 [4914, 2869, 4861, 1700],
 [403, 4336, 3297, 424, 2052, 569],
 [1191, 2410, 1058, 4998, 2235, 642, 2410, 3319, 848, 3574],
 [3024,
  3489,
  1373,
  1977,
  463,
  3200,
  793,
  3952,
  262,
  785,
  895,
  3697,
  3449,
  849,
  1703],
 [1638, 2295, 1573, 3550, 3121, 3745, 3558, 3180, 3571, 1025, 2526],
 [75, 497, 3618, 2002, 3405, 2965, 3200, 4563, 3571, 1025, 2526],
 [1968, 3282, 736, 4485, 1302, 1100, 12, 1048, 3200, 2601],
 [4712, 1348, 4355, 1944, 4526, 3168, 3054, 4387],
 [343, 2187, 4204, 1680, 4457, 4073, 1220, 3408, 4850, 2964, 2529],
 [424, 3227, 1280, 1100, 3200, 3405],
 [3660, 4870, 1213, 2192, 3442, 2221, 4220, 3323, 3525],
 [4250, 4329, 3030, 51, 848, 4336, 817, 3571, 1025, 2526],
 [1304, 1689, 3975, 1717, 3556, 3571, 1025, 2526],
 [497, 1173, 921, 286, 1358, 1131, 987, 4381, 429, 98],
 [4588, 3356, 3955],
 [672, 940, 1338, 4938, 3200, 2164, 3601, 1703],
 [3159, 37

In [22]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [23]:
set_length = 20

In [24]:
embedded = pad_sequences(onehot_repr,padding='pre',maxlen=set_length)
embedded

array([[   0,    0,    0, ...,  341,  667, 1629],
       [   0,    0,    0, ..., 2410, 2109, 1703],
       [   0,    0,    0, ..., 2869, 4861, 1700],
       ...,
       [   0,    0,    0, ..., 3571, 1025, 2526],
       [   0,    0,    0, ..., 3982, 4765, 2987],
       [   0,    0,    0, ..., 1785,  837,  372]])

In [25]:
embedded[0]  #first sentence have 20 length now

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3707,
       3742, 1442, 4891, 1280, 1794, 3427,  341,  667, 1629])

In [27]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [28]:
# Creating model

dim = 40
model = Sequential()
model.add(Embedding(vocab_size, dim, input_length = set_length))
model.add(LSTM(100))                               #train using LSTM - 1 layer with 100 neurons
                                                   #vector produced by embedding layer is given as input to LSTM for training
model.add(Dense(1,activation='sigmoid'))  #dense layer=1 as it is classification problem - 1 final o/p, in which we apply sigmoid 
                                          # activation func-which tells propbability of class 1 or class 2
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [29]:
y

0        1
1        0
2        1
3        1
4        1
        ..
18280    0
18281    0
18282    0
18283    1
18284    1
Name: label, Length: 18285, dtype: int64

In [30]:
import numpy as np
X_final = np.array(embedded)
y_final = np.array(y)

In [33]:
print(X_final.shape)
print(y_final.shape)

(18285, 20)
(18285,)


In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

## Model Training

In [34]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 9s 27ms/step - loss: 0.3473 - accuracy: 0.8355 - val_loss: 0.2035 - val_accuracy: 0.9120
Epoch 2/10
192/192 [==============================] - 6s 30ms/step - loss: 0.1482 - accuracy: 0.9410 - val_loss: 0.1919 - val_accuracy: 0.9181
Epoch 3/10
192/192 [==============================] - 6s 29ms/step - loss: 0.1057 - accuracy: 0.9619 - val_loss: 0.2037 - val_accuracy: 0.9175
Epoch 4/10
192/192 [==============================] - 5s 27ms/step - loss: 0.0719 - accuracy: 0.9746 - val_loss: 0.2454 - val_accuracy: 0.9170
Epoch 5/10
192/192 [==============================] - 5s 25ms/step - loss: 0.0464 - accuracy: 0.9856 - val_loss: 0.3038 - val_accuracy: 0.9155
Epoch 6/10
192/192 [==============================] - 5s 26ms/step - loss: 0.0262 - accuracy: 0.9931 - val_loss: 0.3501 - val_accuracy: 0.9158
Epoch 7/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0166 - accuracy: 0.9957 - val_loss: 0.4012 - val_accuracy: 0.9087

In [35]:
y_pred = model.predict_classes(X_test)

C:\Users\User\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [36]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[3068,  351],
       [ 170, 2446]], dtype=int64)

In [37]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9136702568351284